In [ ]:
# 依存関係
'''
poetry add wheel

poetry add --group dev 'ipykernel' 'ipywidgets'

poetry add --group llm 'openvino'\
    "git+https://github.com/huggingface/optimum-intel.git"\
    "git+https://github.com/openvinotoolkit/nncf.git"
'''

In [ ]:
from pathlib import Path
import logging
import os
from datetime import datetime
from llm_config import LLM_MODELS_CONFIG

import openvino as ov
import nncf


In [2]:
## modelがgemmaの場合は、ログインが必要
def login_huggingface_hub():
    from huggingface_hub import notebook_login, whoami
    try:
        whoami()
        print('Authorization token already provided')
    except OSError:
        notebook_login()
        print('Authorization token has been provided')

In [3]:
nncf.set_log_level(logging.ERROR)

core = ov.Core()

# デバイスの特定
support_devices = core.available_devices
device = 'NPU' if 'NPU' in support_devices else 'CPU'


# ダウンロードするモデル
# NPUならPhi-3-mini-4k-instruct, CPUならgemma-2b-it
model_id = "microsoft/Phi-3-mini-4k-instruct" if device == 'NPU' else "google/gemma-2b-it"

model_id = 'microsoft/Phi-3-mini-4k-instruct'
## modelがgemmaの場合は、HuggingFaceのログインが必要
if model_id == "google/gemma-2b-it":
    login_huggingface_hub()

model_name = model_id.split('/')[-1]

remote_code = LLM_MODELS_CONFIG[model_name]['remote_code']  # Phi-3はTrue

# モデルを保存するディレクトリ
model_dir = Path(f'../../model/{model_name}')
fp16_model_dir = model_dir / "FP16"  # float 16bitモデルの保存先
int8_model_dir = model_dir / "INT8"  # 量子化モデルの保存先(8bit)
# # NPUの要件がINT8以上なので、コメントアウト
# int4_model_dir = model_dir / "INT4"  # 量子化モデルの保存先(4bit)

In [4]:
# # NPUの要件がINT8以上なので、コメントアウト

# # INT4の設定
# compression_configs = {
#     # ここのパラメータは要調整
#     # "sym":          対称量子化の利用
#     # 'group_size':  グループサイズ  (64, 128が無難？)
#     # 'ratio':       量子化後のパラメータの割合  (0.5~0.8で試す)
#     "gemma-2b-it": {
#         "sym": True,
#         "group_size": 64,
#         "ratio": 0.6,
#     },
#     "default": {
#         "sym": False,
#         "group_size": 128,
#         "ratio": 0.8,
#     },
# }

In [ ]:
model_id

In [6]:
# optimum-cliでモデルをopenvino形式でダウンロード
export_command_base = "optimum-cli export openvino --model {} --task text-generation-with-past".format(model_id)

def convert_to_fp16():
    global export_command_base
    export_command = ''
    # すでに存在する場合はスキップ
    if (fp16_model_dir / "openvino_model.xml" ).exists():
        return
    if remote_code:
        # Phi-3のみ
        export_command_base += " --trust-remote-code"
    export_command = export_command_base + " --weight-format fp16"
    export_command += " " + str(fp16_model_dir)
    # モデルのダウンロード開始時間
    start_model_download = datetime.now()
    print('export_command:', export_command)
    os.system(export_command)  # モデルのダウンロード
    # モデルのダウンロード終了時間
    end_model_download = datetime.now() - start_model_download
    print('export done', end_model_download.total_seconds())


def convert_to_int8():
    global export_command_base
    if (int8_model_dir / "openvino_model.xml").exists():
        return
    if remote_code:
        export_command_base += " --trust-remote-code"
    export_command = export_command_base + " --weight-format int8"
    export_command += " " + str(int8_model_dir)
    # モデルのダウンロード開始時間
    start_model_download = datetime.now()
    print('export_command:', export_command)
    os.system(export_command)  # モデルのダウンロード
    # モデルのダウンロード終了時間
    end_model_download = datetime.now() - start_model_download
    print('export done', end_model_download.total_seconds())


# # NPUの要件がINT8以上なので、コメントアウト
# def convert_to_int4():
#     global export_command_base
#     if (int4_model_dir / "openvino_model.xml").exists():
#         return
#     if remote_code:
#         export_command_base += " --trust-remote-code"
#     # 量子化の設定
#     model_compression_params  = compression_configs.get(model_name, compression_configs["default"])
#     export_command = export_command_base + " --weight-format int4"
#     int4_compression_args = " --group-size {} --ratio {}".format(model_compression_params["group_size"], model_compression_params["ratio"])
#     if model_compression_params["sym"]:
#         int4_compression_args += " --sym"
#     export_command += int4_compression_args + " " + str(int4_model_dir)
#     # モデルのダウンロード開始時間
#     start_model_download = datetime.now()
#     print('export_command:', export_command)
#     os.system(export_command)  # モデルのダウンロード
#     # モデルのダウンロード終了時間
#     end_model_download = datetime.now() - start_model_download
#     print('export done', end_model_download.total_seconds())


In [7]:
# # NPUの要件がINT8以上なので、コメントアウト
# convert_to_int4()

In [8]:
convert_to_int8()

In [9]:
convert_to_fp16()

In [ ]:
# モデルが保存されているディレクトリのサイズを確認

fp16_weights = fp16_model_dir / "openvino_model.bin"
int8_weights = int8_model_dir / "openvino_model.bin"
# int4_weights = int4_model_dir / "openvino_model.bin"

if fp16_weights.exists():
    print(f"Size of FP16 model is {fp16_weights.stat().st_size / 1024 / 1024:.2f} MB")

# ファイルのサイズを確認
# for precision, compressed_weights in zip([8, 4], [int8_weights, int4_weights]):
for precision, compressed_weights in zip([8], [int8_weights]):
    if compressed_weights.exists():
        print(f"Size of model with INT{precision} compressed weights is {compressed_weights.stat().st_size / 1024 / 1024:.2f} MB")
    if compressed_weights.exists() and fp16_weights.exists():
        print(f"Compression rate for INT{precision} model: {fp16_weights.stat().st_size / compressed_weights.stat().st_size:.3f}")

In [11]:
# デバイスの選択
support_devices = core.available_devices
device = 'NPU' if 'NPU' in support_devices else 'CPU'  # 実機のNPUが使えればいいのだけれど。。。

In [ ]:
device

In [ ]:
from ipywidgets import widgets

available_models = []
# if int4_model_dir.exists():
#     available_models.append("INT4")
if int8_model_dir.exists():
    available_models.append("INT8")
if fp16_model_dir.exists():
    available_models.append("FP16")

model_to_run = widgets.Dropdown(
    options=available_models,
    value=available_models[0],
    description="Model to run:",
    disabled=False,
)

model_to_run

In [ ]:
model_to_run.value

In [ ]:
from transformers import AutoConfig, AutoTokenizer
from optimum.intel.openvino import OVModelForCausalLM

# if model_to_run.value == "INT4":  # 4bitモデルを使う場合
#     model_dir = int4_model_dir
if model_to_run.value == "INT8":  # 8bitモデルを使う場合
    model_dir = int8_model_dir
else:
    model_dir = fp16_model_dir  # 16bitモデルを使う場合
print(f"Loading model from {model_dir}")

ov_config = {"PERFORMANCE_HINT": "LATENCY", "NUM_STREAMS": "1", "CACHE_DIR": ""}

tok = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)

ov_model = OVModelForCausalLM.from_pretrained(
    model_dir,
    device=device,
    ov_config=ov_config,
    config=AutoConfig.from_pretrained(model_dir, trust_remote_code=True),
    trust_remote_code=True,
)

In [17]:
# system prompt
SYSTEM_PROMPT = '''\
# 指示
あなたは、ユーザーが初対面の相手との会話を補助するためのアシスタントです。
ユーザーと相手の会話を円滑に進めるために、ユーザーの補佐をします。
以下の要素を考慮して、**4つの話題をJSONフォーマット**で提示しなさい。

# 条件
1. 初対面の相手との会話を始めるのに適した話題の提供
2. 会話の流れを考慮して、次の話題を提示
3. 単語レベルの短い文章で話題を提示
4. 4つの話題を提示
5. 出力フォーマットは{{JSON}}形式
6. 出力は単一のJSON

**以下のフォーマットで出力しなさい**

{
  "topics": [
    "話題1",
    "話題2",
    "話題3",
    "話題4"
  ]
}

# 例1
## 入力

自分「○○さん、こんにちは。初めまして。私はAといいます。」
相手「こんにちは。私はBです。よろしくお願いします!」

## 出力
{
  "topics": [
    "年齢",
    "どこ出身",
    "趣味",
    "好きな食べ物"
  ]
}

# 例2
## 入力
自分「○○さんって、どんなことが好きですか？」
相手「音楽が好きで、よくライブに行ってます。」

## 出力
{
  "topics": [
    "好きなアーティスト",
    "休日の過ごし方",
    "好きな音楽のジャンル",
    "最近行ったライブ"
  ]
}

# 例3
## 入力
自分「○○さん、最近見た映画はありますか？」
相手「最近○○って映画を見たんですけど、面白かったですよ。」

## 出力
{
  "topics": [
    "映画のジャンル",
    "主演俳優",
    "好きな映画のジャンル",
    "一番好きな映画"
  ]
}
'''

In [ ]:
# モデルの動作確認
tokenizer_kwargs = LLM_MODELS_CONFIG[model_name].get("tokenizer_kwargs", {})



conversation = '''\
自分: 「初めまして〜。俺AI専攻の人と話すの初めてなんだけどAI専攻ってどんなことしてるの？」

相手: 「AI専攻は機械学習とか画像や音声認識とかLLMについて勉強してるよ」
'''

start_message: str = LLM_MODELS_CONFIG[model_name]['start_message']
prompt_template: str = LLM_MODELS_CONFIG[model_name]['prompt_template']

sys_prompt = start_message.format(
    SYSTEM_PROMPT=SYSTEM_PROMPT
)

prompt = sys_prompt + prompt_template.format(
    user=conversation
)

input_tokens = tok(prompt, return_tensors="pt", **tokenizer_kwargs)

answer = ov_model.generate(
    **input_tokens,
    max_new_tokens=100,
    temperature=0.7,
    do_sample=True
)

ans = tok.batch_decode(answer, skip_special_tokens=True)[0]
print(ans)